In [1]:
!pip install /kaggle/input/uspto-whoosh-reloaded-2-7-5-patched/Whoosh_Reloaded-2.7.5-py2.py3-none-any.whl
import random
import whoosh
import pyarrow.parquet as pq
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import seaborn as sns
import squarify
import numpy as np
import os, os.path
from whoosh import index
from whoosh import qparser
from whoosh.fields import Schema, TEXT, KEYWORD, ID, STORED
from whoosh.analysis import StemmingAnalyzer
from whoosh.qparser import QueryParser
from whoosh.index import create_in, open_dir
import re


Processing /kaggle/input/uspto-whoosh-reloaded-2-7-5-patched/Whoosh_Reloaded-2.7.5-py2.py3-none-any.whl


In [2]:
pd.set_option('display.max_colwidth', None)
tot_df =pd.read_csv("/kaggle/input/cpctot/cpctot.dat", encoding='latin1')
cpc_tot_df=tot_df.fillna(0)

cpc_tot_df

,cpc_code,maj_class,min_class,sub_class,fin_class,det_class,det_code,maj_desc,min_desc,sub_desc,fin_desc,det_desc,cpc_desc
0,A,A,A,A,A,0.0,0.0,HUMAN NECESSITIES,x,x,x,x,HUMAN NECESSITIES
1,A01,A,A01,A01,A01,0.0,0.0,HUMAN NECESSITIES,x,x,x,x,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTING; TRAPPING; FISHING
2,A01B,A,A01,A01B,A01B,0.0,0.0,HUMAN NECESSITIES,x,x,x,x,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS DETAILS OR ACCESSORIES OF AGRICULTURAL MACHINES OR IMPLEMENTS IN GENERAL (making or covering furrows or holes for sowing planting or manuring A01C5/00; soil working for engineering purposes E01 E02 E21; {measuring areas for agricultural purposes G01B})
3,A01B1/00,A,A01,A01B,A01B1,0.0,0.0,HUMAN NECESSITIES,x,x,x,x,Hand tools (edge trimmers for lawns A01G3/06 {; machines for working soil A01B35/00; making hand tools B21D})
4,A01B1/02,A,A01,A01B,A01B1,2.0,1.0,HUMAN NECESSITIES,x,x,x,x,Spades; Shovels {(hand-operated dredgers E02F3/02)}
...,...,...,...,...,...,...,...,...,...,...,...,...,...
260804,Y10T483/1873,Y,Y10,Y10T,Y10T483,1873.0,2.0,GENERAL TAGGING OF CROSS-SECTIONAL TECHNOLOGIES,x,x,x,x,Indexing matrix
260805,Y10T483/1882,Y,Y10,Y10T,Y10T483,1882.0,3.0,GENERAL TAGGING OF CROSS-SECTIONAL TECHNOLOGIES,x,x,x,x,Rotary disc
260806,Y10T483/1891,Y,Y10,Y10T,Y10T483,1891.0,3.0,GENERAL TAGGING OF CROSS-SECTIONAL TECHNOLOGIES,x,x,x,x,Chain or belt
260807,Y10T483/19,Y,Y10,Y10T,Y10T483,19.0,1.0,GENERAL TAGGING OF CROSS-SECTIONAL TECHNOLOGIES,x,x,x,x,Miscellaneous


In [3]:
logical_min_results= (cpc_tot_df.min_class == cpc_tot_df.cpc_code)
logical_min_results

0          True
1          True
2         False
3         False
4         False
          ...  
260804    False
260805    False
260806    False
260807    False
260808    False
Length: 260809, dtype: bool

In [4]:
cpctot_min_Logical_df=cpc_tot_df[logical_min_results ]  
cpctot_min_df=cpctot_min_Logical_df [['min_class','cpc_desc']]
cpctot_min_df

,min_class,cpc_desc
0,A,HUMAN NECESSITIES
1,A01,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTING; TRAPPING; FISHING
2970,A21,BAKING; EDIBLE DOUGHS
3254,A22,BUTCHERING; MEAT TREATMENT; PROCESSING POULTRY OR FISH
3494,A23,FOODS OR FOODSTUFFS; TREATMENT THEREOF NOT COVERED BY OTHER CLASSES
...,...,...
245282,H99,SUBJECT MATTER NOT OTHERWISE PROVIDED FOR IN THIS SECTION
245285,Y,GENERAL TAGGING OF NEW TECHNOLOGICAL DEVELOPMENTS; GENERAL TAGGING OF CROSS-SECTIONAL TECHNOLOGIES SPANNING OVER SEVERAL SECTIONS OF THE IPC; TECHNICAL SUBJECTS COVERED BY FORMER USPC CROSS-REFERENCE ART COLLECTIONS [XRACs] AND DIGESTS
245286,Y02,TECHNOLOGIES OR APPLICATIONS FOR MITIGATION OR ADAPTATION AGAINST CLIMATE CHANGE
245626,Y04,INFORMATION OR COMMUNICATION TECHNOLOGIES HAVING AN IMPACT ON OTHER TECHNOLOGY AREAS


In [5]:
cpctot_min_class_df=cpctot_min_df.rename (columns = {'cpc_desc': 'min_desc'})
cpctot_min_desc_df=cpctot_min_class_df.rename (columns = {'cpc_desc': 'min_desc'})
cpctot_min_class_df

,min_class,min_desc
0,A,HUMAN NECESSITIES
1,A01,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTING; TRAPPING; FISHING
2970,A21,BAKING; EDIBLE DOUGHS
3254,A22,BUTCHERING; MEAT TREATMENT; PROCESSING POULTRY OR FISH
3494,A23,FOODS OR FOODSTUFFS; TREATMENT THEREOF NOT COVERED BY OTHER CLASSES
...,...,...
245282,H99,SUBJECT MATTER NOT OTHERWISE PROVIDED FOR IN THIS SECTION
245285,Y,GENERAL TAGGING OF NEW TECHNOLOGICAL DEVELOPMENTS; GENERAL TAGGING OF CROSS-SECTIONAL TECHNOLOGIES SPANNING OVER SEVERAL SECTIONS OF THE IPC; TECHNICAL SUBJECTS COVERED BY FORMER USPC CROSS-REFERENCE ART COLLECTIONS [XRACs] AND DIGESTS
245286,Y02,TECHNOLOGIES OR APPLICATIONS FOR MITIGATION OR ADAPTATION AGAINST CLIMATE CHANGE
245626,Y04,INFORMATION OR COMMUNICATION TECHNOLOGIES HAVING AN IMPACT ON OTHER TECHNOLOGY AREAS


In [6]:
cpc_tot_df.set_index('min_class', inplace=True)
cpctot_min_desc_df.set_index('min_class', inplace=True)
cpc_tot_df.update(cpctot_min_desc_df)
cpc_tot_df_reset=cpc_tot_df.reset_index(drop = True)
cpc_tot_df_reset

,cpc_code,maj_class,sub_class,fin_class,det_class,det_code,maj_desc,min_desc,sub_desc,fin_desc,det_desc,cpc_desc
0,A,A,A,A,0.0,0.0,HUMAN NECESSITIES,HUMAN NECESSITIES,x,x,x,HUMAN NECESSITIES
1,A01,A,A01,A01,0.0,0.0,HUMAN NECESSITIES,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTING; TRAPPING; FISHING,x,x,x,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTING; TRAPPING; FISHING
2,A01B,A,A01B,A01B,0.0,0.0,HUMAN NECESSITIES,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTING; TRAPPING; FISHING,x,x,x,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS DETAILS OR ACCESSORIES OF AGRICULTURAL MACHINES OR IMPLEMENTS IN GENERAL (making or covering furrows or holes for sowing planting or manuring A01C5/00; soil working for engineering purposes E01 E02 E21; {measuring areas for agricultural purposes G01B})
3,A01B1/00,A,A01B,A01B1,0.0,0.0,HUMAN NECESSITIES,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTING; TRAPPING; FISHING,x,x,x,Hand tools (edge trimmers for lawns A01G3/06 {; machines for working soil A01B35/00; making hand tools B21D})
4,A01B1/02,A,A01B,A01B1,2.0,1.0,HUMAN NECESSITIES,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTING; TRAPPING; FISHING,x,x,x,Spades; Shovels {(hand-operated dredgers E02F3/02)}
...,...,...,...,...,...,...,...,...,...,...,...,...
260804,Y10T483/1873,Y,Y10T,Y10T483,1873.0,2.0,GENERAL TAGGING OF CROSS-SECTIONAL TECHNOLOGIES,TECHNICAL SUBJECTS COVERED BY FORMER USPC,x,x,x,Indexing matrix
260805,Y10T483/1882,Y,Y10T,Y10T483,1882.0,3.0,GENERAL TAGGING OF CROSS-SECTIONAL TECHNOLOGIES,TECHNICAL SUBJECTS COVERED BY FORMER USPC,x,x,x,Rotary disc
260806,Y10T483/1891,Y,Y10T,Y10T483,1891.0,3.0,GENERAL TAGGING OF CROSS-SECTIONAL TECHNOLOGIES,TECHNICAL SUBJECTS COVERED BY FORMER USPC,x,x,x,Chain or belt
260807,Y10T483/19,Y,Y10T,Y10T483,19.0,1.0,GENERAL TAGGING OF CROSS-SECTIONAL TECHNOLOGIES,TECHNICAL SUBJECTS COVERED BY FORMER USPC,x,x,x,Miscellaneous


In [7]:
logical_sub_results= (cpc_tot_df_reset.sub_class == cpc_tot_df_reset.cpc_code)
#logical_sub_results_reset = logical_sub_results.reset_index(drop=True)
logical_sub_results

0          True
1          True
2          True
3         False
4         False
          ...  
260804    False
260805    False
260806    False
260807    False
260808    False
Length: 260809, dtype: bool

In [8]:
cpctot_sub_Logical_df=cpc_tot_df_reset[logical_sub_results ]  
cpctot_sub_df=cpctot_sub_Logical_df [['sub_class','cpc_desc']]
cpctot_sub_df_reset = cpctot_sub_df.reset_index(drop=True)
cpctot_sub_df_reset

,sub_class,cpc_desc
0,A,HUMAN NECESSITIES
1,A01,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTING; TRAPPING; FISHING
2,A01B,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS DETAILS OR ACCESSORIES OF AGRICULTURAL MACHINES OR IMPLEMENTS IN GENERAL (making or covering furrows or holes for sowing planting or manuring A01C5/00; soil working for engineering purposes E01 E02 E21; {measuring areas for agricultural purposes G01B})
3,A01C,PLANTING; SOWING; FERTILISING (parts details or accessories of agricultural machines or implements in general A01B51/00?á-?áA01B75/00)
4,A01D,HARVESTING; MOWING
...,...,...
818,Y04,INFORMATION OR COMMUNICATION TECHNOLOGIES HAVING AN IMPACT ON OTHER TECHNOLOGY AREAS
819,Y04S,SYSTEMS INTEGRATING TECHNOLOGIES RELATED TO POWER NETWORK OPERATION COMMUNICATION OR INFORMATION TECHNOLOGIES FOR IMPROVING THE ELECTRICAL POWER GENERATION TRANSMISSION DISTRIBUTION MANAGEMENT OR USAGE i.e. SMART GRIDS
820,Y10,TECHNICAL SUBJECTS COVERED BY FORMER USPC
821,Y10S,TECHNICAL SUBJECTS COVERED BY FORMER USPC CROSS-REFERENCE ART COLLECTIONS [XRACs] AND DIGESTS


In [9]:
cpctot_sub_class_df=cpctot_sub_df_reset.rename (columns = {'cpc_desc': 'sub_desc'})
#cpc_A_sub_desc_df=cpctot_sub_class_df.rename (columns = {'cpc_desc': 'sub_desc'})
cpctot_sub_class_df

,sub_class,sub_desc
0,A,HUMAN NECESSITIES
1,A01,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTING; TRAPPING; FISHING
2,A01B,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS DETAILS OR ACCESSORIES OF AGRICULTURAL MACHINES OR IMPLEMENTS IN GENERAL (making or covering furrows or holes for sowing planting or manuring A01C5/00; soil working for engineering purposes E01 E02 E21; {measuring areas for agricultural purposes G01B})
3,A01C,PLANTING; SOWING; FERTILISING (parts details or accessories of agricultural machines or implements in general A01B51/00?á-?áA01B75/00)
4,A01D,HARVESTING; MOWING
...,...,...
818,Y04,INFORMATION OR COMMUNICATION TECHNOLOGIES HAVING AN IMPACT ON OTHER TECHNOLOGY AREAS
819,Y04S,SYSTEMS INTEGRATING TECHNOLOGIES RELATED TO POWER NETWORK OPERATION COMMUNICATION OR INFORMATION TECHNOLOGIES FOR IMPROVING THE ELECTRICAL POWER GENERATION TRANSMISSION DISTRIBUTION MANAGEMENT OR USAGE i.e. SMART GRIDS
820,Y10,TECHNICAL SUBJECTS COVERED BY FORMER USPC
821,Y10S,TECHNICAL SUBJECTS COVERED BY FORMER USPC CROSS-REFERENCE ART COLLECTIONS [XRACs] AND DIGESTS


In [10]:
cpc_tot_df_reset.set_index('sub_class', inplace=True)
cpctot_sub_class_df.set_index('sub_class', inplace=True)
cpc_tot_df_reset.update(cpctot_sub_class_df)
cpc_tot_df_reset_reset=cpc_tot_df_reset.reset_index(drop = True)
cpc_tot_df_reset_reset

,cpc_code,maj_class,fin_class,det_class,det_code,maj_desc,min_desc,sub_desc,fin_desc,det_desc,cpc_desc
0,A,A,A,0.0,0.0,HUMAN NECESSITIES,HUMAN NECESSITIES,HUMAN NECESSITIES,x,x,HUMAN NECESSITIES
1,A01,A,A01,0.0,0.0,HUMAN NECESSITIES,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTING; TRAPPING; FISHING,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTING; TRAPPING; FISHING,x,x,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTING; TRAPPING; FISHING
2,A01B,A,A01B,0.0,0.0,HUMAN NECESSITIES,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTING; TRAPPING; FISHING,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS DETAILS OR ACCESSORIES OF AGRICULTURAL MACHINES OR IMPLEMENTS IN GENERAL (making or covering furrows or holes for sowing planting or manuring A01C5/00; soil working for engineering purposes E01 E02 E21; {measuring areas for agricultural purposes G01B}),x,x,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS DETAILS OR ACCESSORIES OF AGRICULTURAL MACHINES OR IMPLEMENTS IN GENERAL (making or covering furrows or holes for sowing planting or manuring A01C5/00; soil working for engineering purposes E01 E02 E21; {measuring areas for agricultural purposes G01B})
3,A01B1/00,A,A01B1,0.0,0.0,HUMAN NECESSITIES,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTING; TRAPPING; FISHING,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS DETAILS OR ACCESSORIES OF AGRICULTURAL MACHINES OR IMPLEMENTS IN GENERAL (making or covering furrows or holes for sowing planting or manuring A01C5/00; soil working for engineering purposes E01 E02 E21; {measuring areas for agricultural purposes G01B}),x,x,Hand tools (edge trimmers for lawns A01G3/06 {; machines for working soil A01B35/00; making hand tools B21D})
4,A01B1/02,A,A01B1,2.0,1.0,HUMAN NECESSITIES,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTING; TRAPPING; FISHING,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS DETAILS OR ACCESSORIES OF AGRICULTURAL MACHINES OR IMPLEMENTS IN GENERAL (making or covering furrows or holes for sowing planting or manuring A01C5/00; soil working for engineering purposes E01 E02 E21; {measuring areas for agricultural purposes G01B}),x,x,Spades; Shovels {(hand-operated dredgers E02F3/02)}
...,...,...,...,...,...,...,...,...,...,...,...
260804,Y10T483/1873,Y,Y10T483,1873.0,2.0,GENERAL TAGGING OF CROSS-SECTIONAL TECHNOLOGIES,TECHNICAL SUBJECTS COVERED BY FORMER USPC,TECHNICAL SUBJECTS COVERED BY FORMER US CLASSIFICATION,x,x,Indexing matrix
260805,Y10T483/1882,Y,Y10T483,1882.0,3.0,GENERAL TAGGING OF CROSS-SECTIONAL TECHNOLOGIES,TECHNICAL SUBJECTS COVERED BY FORMER USPC,TECHNICAL SUBJECTS COVERED BY FORMER US CLASSIFICATION,x,x,Rotary disc
260806,Y10T483/1891,Y,Y10T483,1891.0,3.0,GENERAL TAGGING OF CROSS-SECTIONAL TECHNOLOGIES,TECHNICAL SUBJECTS COVERED BY FORMER USPC,TECHNICAL SUBJECTS COVERED BY FORMER US CLASSIFICATION,x,x,Chain or belt
260807,Y10T483/19,Y,Y10T483,19.0,1.0,GENERAL TAGGING OF CROSS-SECTIONAL TECHNOLOGIES,TECHNICAL SUBJECTS COVERED BY FORMER USPC,TECHNICAL SUBJECTS COVERED BY FORMER US CLASSIFICATION,x,x,Miscellaneous


In [11]:
logical_fin_results= (cpc_tot_df_reset_reset.fin_class == cpc_tot_df_reset_reset.cpc_code)
#logical_fin_results= ((cpc_tot_df_reset_reset.fin_class == cpc_tot_df_reset_reset.cpc_code.str[:5]) & (cpc_tot_df_reset_reset.cpc_code.str[-2:-1] == 0))
logical_fin_results

0          True
1          True
2          True
3         False
4         False
          ...  
260804    False
260805    False
260806    False
260807    False
260808    False
Length: 260809, dtype: bool

In [12]:
cpctot_fin_Logical_df=cpc_tot_df_reset_reset[logical_fin_results ]  
cpctot_fin_df=cpctot_fin_Logical_df [['fin_class','cpc_desc']]
cpctot_fin_df

,fin_class,cpc_desc
0,A,HUMAN NECESSITIES
1,A01,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTING; TRAPPING; FISHING
2,A01B,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS DETAILS OR ACCESSORIES OF AGRICULTURAL MACHINES OR IMPLEMENTS IN GENERAL (making or covering furrows or holes for sowing planting or manuring A01C5/00; soil working for engineering purposes E01 E02 E21; {measuring areas for agricultural purposes G01B})
350,A01C,PLANTING; SOWING; FERTILISING (parts details or accessories of agricultural machines or implements in general A01B51/00?á-?áA01B75/00)
499,A01D,HARVESTING; MOWING
...,...,...
245626,Y04,INFORMATION OR COMMUNICATION TECHNOLOGIES HAVING AN IMPACT ON OTHER TECHNOLOGY AREAS
245627,Y04S,SYSTEMS INTEGRATING TECHNOLOGIES RELATED TO POWER NETWORK OPERATION COMMUNICATION OR INFORMATION TECHNOLOGIES FOR IMPROVING THE ELECTRICAL POWER GENERATION TRANSMISSION DISTRIBUTION MANAGEMENT OR USAGE i.e. SMART GRIDS
245669,Y10,TECHNICAL SUBJECTS COVERED BY FORMER USPC
245670,Y10S,TECHNICAL SUBJECTS COVERED BY FORMER USPC CROSS-REFERENCE ART COLLECTIONS [XRACs] AND DIGESTS


In [13]:
cpctot_fin_class_df=cpctot_fin_df.rename (columns = {'cpc_desc': 'fin_desc'})
cpc_A_fin_desc_df=cpctot_fin_class_df.rename (columns = {'cpc_desc': 'fin_desc'})
cpctot_fin_class_df

,fin_class,fin_desc
0,A,HUMAN NECESSITIES
1,A01,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTING; TRAPPING; FISHING
2,A01B,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS DETAILS OR ACCESSORIES OF AGRICULTURAL MACHINES OR IMPLEMENTS IN GENERAL (making or covering furrows or holes for sowing planting or manuring A01C5/00; soil working for engineering purposes E01 E02 E21; {measuring areas for agricultural purposes G01B})
350,A01C,PLANTING; SOWING; FERTILISING (parts details or accessories of agricultural machines or implements in general A01B51/00?á-?áA01B75/00)
499,A01D,HARVESTING; MOWING
...,...,...
245626,Y04,INFORMATION OR COMMUNICATION TECHNOLOGIES HAVING AN IMPACT ON OTHER TECHNOLOGY AREAS
245627,Y04S,SYSTEMS INTEGRATING TECHNOLOGIES RELATED TO POWER NETWORK OPERATION COMMUNICATION OR INFORMATION TECHNOLOGIES FOR IMPROVING THE ELECTRICAL POWER GENERATION TRANSMISSION DISTRIBUTION MANAGEMENT OR USAGE i.e. SMART GRIDS
245669,Y10,TECHNICAL SUBJECTS COVERED BY FORMER USPC
245670,Y10S,TECHNICAL SUBJECTS COVERED BY FORMER USPC CROSS-REFERENCE ART COLLECTIONS [XRACs] AND DIGESTS


In [14]:
cpc_tot_df_reset_reset.set_index('fin_class', inplace=True)
cpc_tot_df_reset_reset.update(cpctot_fin_class_df.set_index('fin_class'))
cpc_tot_df_reset_reset_reset = cpc_tot_df_reset_reset.reset_index()
cpc_tot_df_reset_reset_reset

,fin_class,cpc_code,maj_class,det_class,det_code,maj_desc,min_desc,sub_desc,fin_desc,det_desc,cpc_desc
0,A,A,A,0.0,0.0,HUMAN NECESSITIES,HUMAN NECESSITIES,HUMAN NECESSITIES,HUMAN NECESSITIES,x,HUMAN NECESSITIES
1,A01,A01,A,0.0,0.0,HUMAN NECESSITIES,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTING; TRAPPING; FISHING,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTING; TRAPPING; FISHING,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTING; TRAPPING; FISHING,x,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTING; TRAPPING; FISHING
2,A01B,A01B,A,0.0,0.0,HUMAN NECESSITIES,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTING; TRAPPING; FISHING,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS DETAILS OR ACCESSORIES OF AGRICULTURAL MACHINES OR IMPLEMENTS IN GENERAL (making or covering furrows or holes for sowing planting or manuring A01C5/00; soil working for engineering purposes E01 E02 E21; {measuring areas for agricultural purposes G01B}),SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS DETAILS OR ACCESSORIES OF AGRICULTURAL MACHINES OR IMPLEMENTS IN GENERAL (making or covering furrows or holes for sowing planting or manuring A01C5/00; soil working for engineering purposes E01 E02 E21; {measuring areas for agricultural purposes G01B}),x,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS DETAILS OR ACCESSORIES OF AGRICULTURAL MACHINES OR IMPLEMENTS IN GENERAL (making or covering furrows or holes for sowing planting or manuring A01C5/00; soil working for engineering purposes E01 E02 E21; {measuring areas for agricultural purposes G01B})
3,A01B1,A01B1/00,A,0.0,0.0,HUMAN NECESSITIES,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTING; TRAPPING; FISHING,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS DETAILS OR ACCESSORIES OF AGRICULTURAL MACHINES OR IMPLEMENTS IN GENERAL (making or covering furrows or holes for sowing planting or manuring A01C5/00; soil working for engineering purposes E01 E02 E21; {measuring areas for agricultural purposes G01B}),x,x,Hand tools (edge trimmers for lawns A01G3/06 {; machines for working soil A01B35/00; making hand tools B21D})
4,A01B1,A01B1/02,A,2.0,1.0,HUMAN NECESSITIES,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTING; TRAPPING; FISHING,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS DETAILS OR ACCESSORIES OF AGRICULTURAL MACHINES OR IMPLEMENTS IN GENERAL (making or covering furrows or holes for sowing planting or manuring A01C5/00; soil working for engineering purposes E01 E02 E21; {measuring areas for agricultural purposes G01B}),x,x,Spades; Shovels {(hand-operated dredgers E02F3/02)}
...,...,...,...,...,...,...,...,...,...,...,...
260804,Y10T483,Y10T483/1873,Y,1873.0,2.0,GENERAL TAGGING OF CROSS-SECTIONAL TECHNOLOGIES,TECHNICAL SUBJECTS COVERED BY FORMER USPC,TECHNICAL SUBJECTS COVERED BY FORMER US CLASSIFICATION,x,x,Indexing matrix
260805,Y10T483,Y10T483/1882,Y,1882.0,3.0,GENERAL TAGGING OF CROSS-SECTIONAL TECHNOLOGIES,TECHNICAL SUBJECTS COVERED BY FORMER USPC,TECHNICAL SUBJECTS COVERED BY FORMER US CLASSIFICATION,x,x,Rotary disc
260806,Y10T483,Y10T483/1891,Y,1891.0,3.0,GENERAL TAGGING OF CROSS-SECTIONAL TECHNOLOGIES,TECHNICAL SUBJECTS COVERED BY FORMER USPC,TECHNICAL SUBJECTS COVERED BY FORMER US CLASSIFICATION,x,x,Chain or belt
260807,Y10T483,Y10T483/19,Y,19.0,1.0,GENERAL TAGGING OF CROSS-SECTIONAL TECHNOLOGIES,TECHNICAL SUBJECTS COVERED BY FORMER USPC,TECHNICAL SUBJECTS COVERED BY FORMER US CLASSIFICATION,x,x,Miscellaneous


In [15]:
cpctot_fin_Logical_df=cpc_tot_df_reset_reset_reset[(cpc_tot_df_reset_reset_reset['cpc_code'].str.find('/')>4) & (cpc_tot_df_reset_reset_reset['det_class']==0)]
cpctot_fin_Logical_df

,fin_class,cpc_code,maj_class,det_class,det_code,maj_desc,min_desc,sub_desc,fin_desc,det_desc,cpc_desc
3,A01B1,A01B1/00,A,0.0,0.0,HUMAN NECESSITIES,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTING; TRAPPING; FISHING,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS DETAILS OR ACCESSORIES OF AGRICULTURAL MACHINES OR IMPLEMENTS IN GENERAL (making or covering furrows or holes for sowing planting or manuring A01C5/00; soil working for engineering purposes E01 E02 E21; {measuring areas for agricultural purposes G01B}),x,x,Hand tools (edge trimmers for lawns A01G3/06 {; machines for working soil A01B35/00; making hand tools B21D})
27,A01B3,A01B3/00,A,0.0,0.0,HUMAN NECESSITIES,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTING; TRAPPING; FISHING,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS DETAILS OR ACCESSORIES OF AGRICULTURAL MACHINES OR IMPLEMENTS IN GENERAL (making or covering furrows or holes for sowing planting or manuring A01C5/00; soil working for engineering purposes E01 E02 E21; {measuring areas for agricultural purposes G01B}),x,x,Ploughs with fixed plough-shares
71,A01B5,A01B5/00,A,0.0,0.0,HUMAN NECESSITIES,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTING; TRAPPING; FISHING,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS DETAILS OR ACCESSORIES OF AGRICULTURAL MACHINES OR IMPLEMENTS IN GENERAL (making or covering furrows or holes for sowing planting or manuring A01C5/00; soil working for engineering purposes E01 E02 E21; {measuring areas for agricultural purposes G01B}),x,x,Ploughs with rolling non-driven tools e.g. discs (with rotary driven tools A01B9/00)
80,A01B7,A01B7/00,A,0.0,0.0,HUMAN NECESSITIES,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTING; TRAPPING; FISHING,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS DETAILS OR ACCESSORIES OF AGRICULTURAL MACHINES OR IMPLEMENTS IN GENERAL (making or covering furrows or holes for sowing planting or manuring A01C5/00; soil working for engineering purposes E01 E02 E21; {measuring areas for agricultural purposes G01B}),x,x,Disc-like soil-working implements usable either as ploughs or as harrows or the like
81,A01B9,A01B9/00,A,0.0,0.0,HUMAN NECESSITIES,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTING; TRAPPING; FISHING,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS DETAILS OR ACCESSORIES OF AGRICULTURAL MACHINES OR IMPLEMENTS IN GENERAL (making or covering furrows or holes for sowing planting or manuring A01C5/00; soil working for engineering purposes E01 E02 E21; {measuring areas for agricultural purposes G01B}),x,x,Ploughs with rotary driven tools (tilling implements with rotary driven tools A01B33/00)
...,...,...,...,...,...,...,...,...,...,...,...
259523,Y10T428,Y10T428/00,Y,0.0,0.0,GENERAL TAGGING OF CROSS-SECTIONAL TECHNOLOGIES,TECHNICAL SUBJECTS COVERED BY FORMER USPC,TECHNICAL SUBJECTS COVERED BY FORMER US CLASSIFICATION,x,x,Stock material or miscellaneous articles
260227,Y10T436,Y10T436/00,Y,0.0,0.0,GENERAL TAGGING OF CROSS-SECTIONAL TECHNOLOGIES,TECHNICAL SUBJECTS COVERED BY FORMER USPC,TECHNICAL SUBJECTS COVERED BY FORMER US CLASSIFICATION,x,x,Chemistry: analytical and immunological testing
260319,Y10T442,Y10T442/00,Y,0.0,0.0,GENERAL TAGGING OF CROSS-SECTIONAL TECHNOLOGIES,TECHNICAL SUBJECTS COVERED BY FORMER USPC,TECHNICAL SUBJECTS COVERED BY FORMER US CLASSIFICATION,x,x,Fabric [woven knitted or nonwoven textile or cloth etc.]
260737,Y10T464,Y10T464/00,Y,0.0,0.0,GENERAL TAGGING OF CROSS-SECTIONAL TECHNOLOGIES,TECHNICAL SUBJECTS COVERED BY FORMER USPC,TECHNICAL SUBJECTS COVERED BY FORMER US CLASSIFICATION,x,x,Rotary shafts gudgeons housings and flexible couplings for rotary shafts


In [16]:
cpctot_fin_df=cpctot_fin_Logical_df [['fin_class','cpc_desc']]
cpctot_fin_df

,fin_class,cpc_desc
3,A01B1,Hand tools (edge trimmers for lawns A01G3/06 {; machines for working soil A01B35/00; making hand tools B21D})
27,A01B3,Ploughs with fixed plough-shares
71,A01B5,Ploughs with rolling non-driven tools e.g. discs (with rotary driven tools A01B9/00)
80,A01B7,Disc-like soil-working implements usable either as ploughs or as harrows or the like
81,A01B9,Ploughs with rotary driven tools (tilling implements with rotary driven tools A01B33/00)
...,...,...
259523,Y10T428,Stock material or miscellaneous articles
260227,Y10T436,Chemistry: analytical and immunological testing
260319,Y10T442,Fabric [woven knitted or nonwoven textile or cloth etc.]
260737,Y10T464,Rotary shafts gudgeons housings and flexible couplings for rotary shafts


In [17]:
cpctot_fin_class_df=cpctot_fin_df.rename (columns = {'cpc_desc': 'fin_desc'})
cpctot_fin_class_df

,fin_class,fin_desc
3,A01B1,Hand tools (edge trimmers for lawns A01G3/06 {; machines for working soil A01B35/00; making hand tools B21D})
27,A01B3,Ploughs with fixed plough-shares
71,A01B5,Ploughs with rolling non-driven tools e.g. discs (with rotary driven tools A01B9/00)
80,A01B7,Disc-like soil-working implements usable either as ploughs or as harrows or the like
81,A01B9,Ploughs with rotary driven tools (tilling implements with rotary driven tools A01B33/00)
...,...,...
259523,Y10T428,Stock material or miscellaneous articles
260227,Y10T436,Chemistry: analytical and immunological testing
260319,Y10T442,Fabric [woven knitted or nonwoven textile or cloth etc.]
260737,Y10T464,Rotary shafts gudgeons housings and flexible couplings for rotary shafts


In [18]:
#cpc_tot_df_reset_reset_reset.set_index('fin_class', inplace=True)
cpctot_fin_class_df.set_index('fin_class', inplace=True)
cpc_tot_df_reset_reset_reset.update(cpctot_fin_class_df)
cpc_tot_df_reset_reset_reset_reset = cpc_tot_df_reset_reset_reset.reset_index()
del cpc_tot_df_reset_reset_reset_reset['det_desc']
cons_cpc_tot_df=cpc_tot_df_reset_reset_reset_reset[['cpc_code','cpc_desc']]
cons_cpc_tot_df

,cpc_code,cpc_desc
0,A,HUMAN NECESSITIES
1,A01,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTING; TRAPPING; FISHING
2,A01B,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS DETAILS OR ACCESSORIES OF AGRICULTURAL MACHINES OR IMPLEMENTS IN GENERAL (making or covering furrows or holes for sowing planting or manuring A01C5/00; soil working for engineering purposes E01 E02 E21; {measuring areas for agricultural purposes G01B})
3,A01B1/00,Hand tools (edge trimmers for lawns A01G3/06 {; machines for working soil A01B35/00; making hand tools B21D})
4,A01B1/02,Spades; Shovels {(hand-operated dredgers E02F3/02)}
...,...,...
260804,Y10T483/1873,Indexing matrix
260805,Y10T483/1882,Rotary disc
260806,Y10T483/1891,Chain or belt
260807,Y10T483/19,Miscellaneous


In [19]:
cons_cpc_tot_df.to_csv('cons_cpc_tot.csv')

In [20]:
file_path = "/kaggle/input/uspto-explainable-ai/patent_metadata.parquet"
metadata_df = pd.read_parquet(file_path)

In [21]:
publications_df=metadata_df.reset_index()
exploded_codes_df=publications_df.explode('cpc_codes')
all_publications_df=exploded_codes_df.rename (columns = {'cpc_codes': 'cpc_code'})
all_publications_df

,index,publication_number,publication_date,filing_date,family_id,cpc_code
0,0,US-1-A,1836-07-13,NaT,2060279.0,B61C11/04
1,1,US-1-P,1931-08-18,1930-08-06,49893162.0,A01H5/02
1,1,US-1-P,1931-08-18,1930-08-06,49893162.0,A01H6/749
2,2,US-10-A,1836-08-10,NaT,2060288.0,B27J1/00
3,3,US-1000-A,1838-11-03,NaT,2061286.0,B60G11/04
...,...,...,...,...,...,...
13307746,13307746,US-T999003-I4,1980-10-07,1980-04-25,3067331.0,G03C8/10
13307747,13307747,US-X1-I1,1790-07-31,1790-07-31,40303722.0,C01D3/08
13307748,13307748,US-X7668-I1,1833-07-22,NaT,53773065.0,NaN
13307749,13307749,US-X7982-I1,1834-01-27,NaT,53773067.0,NaN


In [22]:
all_publications_df.dropna(subset=["cpc_code"], inplace=True)
trunc_publications=all_publications_df.copy()
trunc_publications

,index,publication_number,publication_date,filing_date,family_id,cpc_code
0,0,US-1-A,1836-07-13,NaT,2060279.0,B61C11/04
1,1,US-1-P,1931-08-18,1930-08-06,49893162.0,A01H5/02
1,1,US-1-P,1931-08-18,1930-08-06,49893162.0,A01H6/749
2,2,US-10-A,1836-08-10,NaT,2060288.0,B27J1/00
3,3,US-1000-A,1838-11-03,NaT,2061286.0,B60G11/04
...,...,...,...,...,...,...
13307743,13307743,US-T998014-I4,1980-09-02,1979-03-19,26695056.0,G03C7/344
13307744,13307744,US-T999001-I4,1980-10-07,1979-07-24,10503881.0,B65G57/10
13307745,13307745,US-T999002-I4,1980-10-07,1979-10-02,22162911.0,B05D7/26
13307746,13307746,US-T999003-I4,1980-10-07,1980-04-25,3067331.0,G03C8/10


In [23]:
del trunc_publications['filing_date']
trunc_publications

,index,publication_number,publication_date,family_id,cpc_code
0,0,US-1-A,1836-07-13,2060279.0,B61C11/04
1,1,US-1-P,1931-08-18,49893162.0,A01H5/02
1,1,US-1-P,1931-08-18,49893162.0,A01H6/749
2,2,US-10-A,1836-08-10,2060288.0,B27J1/00
3,3,US-1000-A,1838-11-03,2061286.0,B60G11/04
...,...,...,...,...,...
13307743,13307743,US-T998014-I4,1980-09-02,26695056.0,G03C7/344
13307744,13307744,US-T999001-I4,1980-10-07,10503881.0,B65G57/10
13307745,13307745,US-T999002-I4,1980-10-07,22162911.0,B05D7/26
13307746,13307746,US-T999003-I4,1980-10-07,3067331.0,G03C8/10


In [24]:
del trunc_publications['publication_date']

trunc_publications

,index,publication_number,family_id,cpc_code
0,0,US-1-A,2060279.0,B61C11/04
1,1,US-1-P,49893162.0,A01H5/02
1,1,US-1-P,49893162.0,A01H6/749
2,2,US-10-A,2060288.0,B27J1/00
3,3,US-1000-A,2061286.0,B60G11/04
...,...,...,...,...
13307743,13307743,US-T998014-I4,26695056.0,G03C7/344
13307744,13307744,US-T999001-I4,10503881.0,B65G57/10
13307745,13307745,US-T999002-I4,22162911.0,B05D7/26
13307746,13307746,US-T999003-I4,3067331.0,G03C8/10


In [25]:
del trunc_publications['family_id']
trunc_publications

,index,publication_number,cpc_code
0,0,US-1-A,B61C11/04
1,1,US-1-P,A01H5/02
1,1,US-1-P,A01H6/749
2,2,US-10-A,B27J1/00
3,3,US-1000-A,B60G11/04
...,...,...,...
13307743,13307743,US-T998014-I4,G03C7/344
13307744,13307744,US-T999001-I4,B65G57/10
13307745,13307745,US-T999002-I4,B05D7/26
13307746,13307746,US-T999003-I4,G03C8/10


In [26]:
del trunc_publications['index']
trunc_publications

,publication_number,cpc_code
0,US-1-A,B61C11/04
1,US-1-P,A01H5/02
1,US-1-P,A01H6/749
2,US-10-A,B27J1/00
3,US-1000-A,B60G11/04
...,...,...
13307743,US-T998014-I4,G03C7/344
13307744,US-T999001-I4,B65G57/10
13307745,US-T999002-I4,B05D7/26
13307746,US-T999003-I4,G03C8/10


In [27]:
import pandas as pd

# Read the JSON file into a DataFrame
jdf = pd.read_json('/kaggle/input/uspto-explainable-ai/train_index_patent_ids.json')

# Assuming `train_df` is supposed to be your DataFrame, define it or read it from somewhere
train_df = pd.DataFrame()

# Assign values from jdf to the `publication_number` column of train_df
train_df['publication_number'] = jdf[:]

# Reset the index of train_df
train_df.reset_index(drop=True, inplace=True)

train_df


,publication_number
0,US-10000095-B2
1,US-10000099-B2
2,US-10000109-B2
3,US-10000176-B2
4,US-10000181-B2
...,...
199995,US-T973016-I4
199996,US-T979006-I4
199997,US-T984005-I4
199998,US-T993001-I4


In [28]:
test_df = pd.read_csv('/kaggle/input/uspto-explainable-ai/test.csv', header = None)

In [29]:
test_df.reset_index(drop=True)
#testdf= newdf[51:]
test_data=test_df.reset_index(drop=True)
test_data_df=test_data.loc[1:,:0,]
test_data_df=test_data_df.rename(columns={0: 'publication_number'})
test_data_df

,publication_number
1,US-2017082634-A1
2,US-2017180470-A1
3,US-2018029544-A1
4,US-2022408153-A1
5,US-2268569-A
6,US-3371854-A
7,US-3589189-A
8,US-3881203-A
9,US-4845770-A
10,US-695233-A


In [30]:
#test_add=pd.merge(all_publications_df,test_data_df,on='publication_number')
total_df=pd.concat([train_df,test_data_df])
total_df.to_csv('traintest.csv')
total_df

,publication_number
0,US-10000095-B2
1,US-10000099-B2
2,US-10000109-B2
3,US-10000176-B2
4,US-10000181-B2
...,...
6,US-3371854-A
7,US-3589189-A
8,US-3881203-A
9,US-4845770-A


In [31]:
train_add=pd.merge(trunc_publications,train_df,on='publication_number')
train_add

,publication_number,cpc_code
0,US-10000095-B2,B60C13/02
1,US-10000095-B2,B60C15/0045
2,US-10000095-B2,B60C15/0242
3,US-10000095-B2,B60C17/0009
4,US-10000095-B2,B60C19/00
...,...,...
1332528,US-T979006-I4,H01L29/4238
1332529,US-T979006-I4,H01L29/78
1332530,US-T984005-I4,G03C7/36
1332531,US-T993001-I4,D06F39/024


In [32]:
test_add=pd.merge(trunc_publications,test_data_df,on='publication_number')
test_add

,publication_number,cpc_code
0,US-2017082634-A1,C12Q1/485
1,US-2017082634-A1,G01N2570/00
2,US-2017082634-A1,G01N33/6845
3,US-2017082634-A1,G01N33/6848
4,US-2017082634-A1,G06F19/20
...,...,...
56,US-3589189-A,G01F3/226
57,US-3881203-A,B42F9/008
58,US-4845770-A,G06K7/10861
59,US-4845770-A,G06V30/224


In [33]:
sorted_add=pd.merge(trunc_publications,total_df,on='publication_number')
sorted_add

,publication_number,cpc_code
0,US-10000095-B2,B60C13/02
1,US-10000095-B2,B60C15/0045
2,US-10000095-B2,B60C15/0242
3,US-10000095-B2,B60C17/0009
4,US-10000095-B2,B60C19/00
...,...,...
1332589,US-T979006-I4,H01L29/4238
1332590,US-T979006-I4,H01L29/78
1332591,US-T984005-I4,G03C7/36
1332592,US-T993001-I4,D06F39/024


In [34]:
merged_train_df=pd.merge(train_add,cons_cpc_tot_df,on='cpc_code')
merged_train_df.to_csv('merged_train.csv')
merged_train_df

,publication_number,cpc_code,cpc_desc
0,US-10000095-B2,B60C13/02,Arrangement of grooves or ribs
1,US-10000095-B2,B60C15/0045,{with ply turn-up up to the belt edges i.e. folded around the bead core and extending to the belt edges}
2,US-10000095-B2,B60C15/0242,{with bead extensions located radially outside the rim flange position e.g. rim flange protectors}
3,US-10000095-B2,B60C17/0009,{comprising sidewall rubber inserts e.g. crescent shaped inserts}
4,US-10000095-B2,B60C19/00,Tyre parts or constructions not otherwise provided for
...,...,...,...
1291722,US-T979006-I4,H01L29/4238,{characterised by the surface lay-out}
1291723,US-T979006-I4,H01L29/78,with field effect produced by an insulated gate {(H01L29/7725 H01L29/775 H01L29/778 take precedence)}
1291724,US-T984005-I4,G03C7/36,Couplers containing compounds with active methylene groups (G03C7/327 G03C7/333 take precedence)
1291725,US-T993001-I4,D06F39/024,{mounted on the agitator or the rotating drum; Free body dispensers}


In [35]:
merged_test_df=pd.merge(test_add,cons_cpc_tot_df,on='cpc_code')
merged_test_df.to_csv('merged_test.csv')
merged_test_df

,publication_number,cpc_code,cpc_desc
0,US-2017082634-A1,C12Q1/485,{involving kinase}
1,US-2017082634-A1,G01N2570/00,Omics e.g. proteomics glycomics or lipidomics; Methods of analysis focusing on the entire complement of classes of biological molecules or subsets thereof i.e. focusing on proteomes glycomes or lipidomes
2,US-2017082634-A1,G01N33/6845,{Methods of identifying protein-protein interactions in protein mixtures}
3,US-2017082634-A1,G01N33/6848,{Methods of protein analysis involving mass spectrometry}
4,US-2017082634-A1,G16B25/00,ICT specially adapted for hybridisation; ICT specially adapted for gene or protein expression
5,US-2017082634-A1,G16B25/10,Gene or protein expression profiling; Expression-ratio estimation or normalisation
6,US-2017082634-A1,G16B5/00,ICT specially adapted for modelling or simulations in systems biology e.g. gene-regulatory networks protein interaction networks or metabolic networks
7,US-2017180470-A1,H04L67/1063,{Discovery through centralising entities}
8,US-2017180470-A1,H04L67/1074,for supporting data block transmission mechanisms (file transfer H04L67/06)
9,US-2017180470-A1,H04L67/1097,for distributed storage of data in networks e.g. transport arrangements for network file system [NFS] storage area networks [SAN] or network attached storage [NAS]


In [36]:
merged_total_df=pd.merge(sorted_add,cons_cpc_tot_df,on='cpc_code')
merged_total_df

,publication_number,cpc_code,cpc_desc
0,US-10000095-B2,B60C13/02,Arrangement of grooves or ribs
1,US-10000095-B2,B60C15/0045,{with ply turn-up up to the belt edges i.e. folded around the bead core and extending to the belt edges}
2,US-10000095-B2,B60C15/0242,{with bead extensions located radially outside the rim flange position e.g. rim flange protectors}
3,US-10000095-B2,B60C17/0009,{comprising sidewall rubber inserts e.g. crescent shaped inserts}
4,US-10000095-B2,B60C19/00,Tyre parts or constructions not otherwise provided for
...,...,...,...
1291780,US-T979006-I4,H01L29/4238,{characterised by the surface lay-out}
1291781,US-T979006-I4,H01L29/78,with field effect produced by an insulated gate {(H01L29/7725 H01L29/775 H01L29/778 take precedence)}
1291782,US-T984005-I4,G03C7/36,Couplers containing compounds with active methylene groups (G03C7/327 G03C7/333 take precedence)
1291783,US-T993001-I4,D06F39/024,{mounted on the agitator or the rotating drum; Free body dispensers}


In [37]:
pd.set_option('display.max_colwidth', None)
ttnearest_df = pd.DataFrame()
ttnearest_df = pd.read_csv('/kaggle/input/ttnearest-condensed/ttnearest_consolidated.csv')
ttnearest_df

,publication_number,nearest_numbers
0,US-10000095-B2,US-9579932-B2 US-9475347-B2 US-2014238571-A1 US-10661611-B2 US-9381776-B2 US-10744826-B2 US-10821785-B2 US-10315467-B2 US-9493040-B2 US-10081220-B2 US-11383560-B2 US-11260704-B2 US-10513154-B2 US-2012060994-A1 US-2017197471-A1 US-10328753-B2 US-11279179-B2 US-11173756-B2 US-10632798-B2 US-11040581-B2 US-9975385-B2 US-10688835-B2 US-9272583-B2 US-11207929-B2 US-10596860-B2 US-9849732-B2 US-11400767-B2 US-2017225523-A1 US-9956828-B2 US-10870318-B2 US-2020016941-A1 US-2011126956-A1 US-2011030862-A1 US-10870317-B2 US-9656521-B2 US-10040322-B2 US-2022161609-A1 US-11198333-B2 US-9931895-B2 US-10589577-B2 US-10946702-B2 US-9849733-B2 US-9902200-B2 US-10052916-B2 US-11400768-B2 US-7712498-B2 US-11667160-B2 US-9815338-B2 US-11331963-B2 US-2017057300-A1
1,US-10000099-B2,US-9313932-B2 US-6647772-B2 US-2004263324-A1 US-10021469-B2 US-7748717-B2 US-2012188069-A1 US-8210037-B2 US-9185831-B2 US-7061146-B2 US-2019218831-A1 US-7451635-B2 US-2020190870-A1 US-7997422-B2 US-9497868-B2 US-2012251782-A1 US-11203236-B2 US-7615902-B2 US-7929377-B2 US-8156797-B2 US-10024433-B2 US-2021053402-A1 US-8490480-B2 US-2019226248-A1 US-11122700-B2 US-7926526-B2 US-7752753-B2 US-2011127995-A1 US-10479160-B2 US-6922141-B2 US-2009134866-A1 US-2022332027-A1 US-7798008-B2 US-11096295-B2 US-2016186807-A1 US-10939565-B2 US-2020294481-A1 US-2019242454-A1 US-2011169251-A1 US-6297448-B1 US-7487668-B2 US-2006125612-A1 US-10933505-B2 US-9673432-B2 US-10285284-B2 US-10785881-B2 US-2020256095-A1 US-2019194983-A1 US-9211770-B2 US-9190766-B2 US-9731749-B2
2,US-10000109-B2,US-10967699-B2 US-6837061-B2 US-9170038-B2 US-9562712-B2 US-10226980-B2 US-9630474-B2 US-2016031292-A1 US-10254027-B2 US-9452659-B2 US-10759259-B2 US-2012312050-A1 US-2015059367-A1 US-10562372-B2 US-9212838-B2 US-7607313-B2 US-10571165-B2 US-11413931-B2 US-9671145-B2 US-2019101313-A1 US-2017191716-A1 US-10525792-B2 US-7290400-B2 US-6758052-B2 US-2021086585-A1 US-11383584-B2 US-9810137-B2 US-11247534-B2 US-6898944-B2 US-8752398-B2 US-7007856-B2 US-9643468-B2 US-2016209100-A1 US-7131281-B2 US-11476689-B2 US-6655164-B2 US-10907879-B2 US-11364767-B2 US-2010175406-A1 US-9885319-B2 US-10556487-B2 US-11479082-B2 US-11267315-B2 US-11421925-B2 US-10336193-B2 US-7063138-B2 US-10369864-B2 US-11338648-B2 US-9829242-B2 US-11624531-B2 US-9914338-B2
3,US-10000176-B2,US-7213837-B2 US-9428135-B1 US-9527468-B2 US-6092836-A US-8360469-B2 US-10906497-B2 US-8371609-B2 US-2005212269-A1 US-9254809-B2 US-9650010-B2 US-10710543-B2 US-8840137-B2 US-2023001880-A1 US-8047564-B2 US-5899489-A US-2013015642-A1 US-8414020-B2 US-5676394-A US-5498027-A US-9067558-B2 US-2019023216-A1 US-7578520-B2 US-5997032-A US-9586554-B1 US-8651524-B2 US-11285903-B2 US-6299197-B1 US-11535185-B2 US-10434971-B2 US-5938232-A US-11396269-B2 US-8662530-B2 US-2012074677-A1 US-6443484-B2 US-10696266-B2 US-10471918-B2 US-2004075250-A1 US-7204510-B2 US-10668887-B2 US-11332096-B2 US-9067566-B1 US-2010032929-A1 US-8708365-B2 US-6942242-B2 US-10059298-B2 US-11214217-B2 US-2021101559-A1 US-11608021-B2 US-8789849-B2 US-7770919-B2
4,US-10000181-B2,US-9126555-B2 US-9090221-B2 US-10525931-B2 US-10399539-B2 US-7887096-B2 US-10703332-B2 US-10730475-B2 US-8052219-B2 US-9950690-B2 US-7137648-B2 US-8585090-B2 US-6840544-B2 US-9988012-B2 US-8678510-B2 US-2019135226-A1 US-8226122-B2 US-2005012319-A1 US-10744974-B2 US-9908505-B2 US-8132829-B2 US-7350734-B2 US-2013193740-A1 US-2014265288-A1 US-8641097-B2 US-8550499-B2 US-10246047-B2 US-11007975-B2 US-7152824-B2 US-10625709-B2 US-6336606-B1 US-10300883-B2 US-9457764-B2 US-9849859-B2 US-11192520-B2 US-8393644-B2 US-10286872-B2 US-10543807-B2 US-6688646-B2 US-2013299620-A1 US-9204691-B2 US-7168742-B2 US-11325561-B2 US-11014530-B2 US-7360795-B2 US-6425542-B2 US-9821760-B2 US-2011133439-A1 US-2010096902-A1 US-5671894-A US-8528987-B2
...,...,...
200069,US-T940011-I4,US-1434892-A US-3420913-A US-1900663-A US-3668253-A US-3226312-A US-1880528-A US-T900022-I4 US-1793161-A US-RE28234-E US-

In [38]:
corrected_df=pd.DataFrame()
corrected_df =   pd.read_csv('/kaggle/input/corrected-merge-traintest/corrected_merge_traintest.csv')
corrected_df

,idx,publication_number,cpc_code,cpc_desc
0,0,US-10000095-B2,B60C13/02,"[ribs, grooves, arrangement]"
1,1,US-10000095-B2,B60C15/0045,"[edges, bead, turn, ply, belt, extending, core, folded]"
2,2,US-10000095-B2,B60C15/0242,"[flange, position, rim, protectors, bead, located, outside, extensions, radially]"
3,3,US-10000095-B2,B60C17/0009,"[shaped, sidewall, crescent, comprising, inserts, rubber]"
4,4,US-10000095-B2,B60C19/00,"[tyre, constructions, parts, provided]"
...,...,...,...,...
1048570,1048570,US-7875615-B2,A61P15/12,"[climacteric, disorders]"
1048571,1048571,US-7875615-B2,A61P25/00,"[nervous, disorders, drugs]"
1048572,1048572,US-7875615-B2,A61P35/00,"[antineoplastic, agents]"
1048573,1048573,US-7875615-B2,C07D249/08,"[hydrogenated, triazoles]"


In [39]:
last_sub_heading_code = corrected_df[corrected_df['cpc_code'].str.endswith('/00')]
last_sub_heading_code

,idx,publication_number,cpc_code,cpc_desc
4,4,US-10000095-B2,B60C19/00,"[tyre, constructions, parts, provided]"
43,43,US-10000237-B2,B60W2554/00,"[input, parameters, objects, relating]"
50,50,US-10000237-B2,B62D6/00,"[conditions, depending, automatically, vehicle, responded, wheels, sensed, direction, steering, arrangements, means, controlling, combined, body, initiating, control, inclining, driving, change, valves, circuits, bends]"
71,71,US-10000398-B2,E02F1/00,"[methods, making, soil, shifting, dredgers, machines, mining, general, embankments, working]"
102,102,US-10000708-B2,C10G7/00,"[distillation, hydrocarbon, oils]"
...,...,...,...,...
1048538,1048538,US-7875257-B2,B82Y30/00,"[nanotechnology, nanocomposites, surface, science, materials]"
1048539,1048539,US-7875257-B2,B82Y40/00,"[nanostructures, manufacture, treatment]"
1048569,1048569,US-7875615-B2,A61P15/00,"[hormones, disorders, sexual, contraceptives, drugs, genital, sex]"
1048571,1048571,US-7875615-B2,A61P25/00,"[nervous, disorders, drugs]"


In [40]:
only_cpc_code = pd.merge(corrected_df, last_sub_heading_code, how='left', indicator=True)
only_cpc_code = only_cpc_code[only_cpc_code['_merge'] == 'left_only'].drop('_merge', axis=1)
only_cpc_code

,idx,publication_number,cpc_code,cpc_desc
0,0,US-10000095-B2,B60C13/02,"[ribs, grooves, arrangement]"
1,1,US-10000095-B2,B60C15/0045,"[edges, bead, turn, ply, belt, extending, core, folded]"
2,2,US-10000095-B2,B60C15/0242,"[flange, position, rim, protectors, bead, located, outside, extensions, radially]"
3,3,US-10000095-B2,B60C17/0009,"[shaped, sidewall, crescent, comprising, inserts, rubber]"
5,5,US-10000095-B2,B60C2015/009,"[value, ratio, carcass, portion, proportion, defined, terminal, height, section, numerical, terms]"
...,...,...,...,...
1048567,1048567,US-7875427-B2,G01N33/6863,"[growth, gm, cytokines, cell, cnf, csf, biological, lymphotoxin, mif, response, proteins, modifying, differentiation, proliferation, immune, tnf, receptors]"
1048568,1048568,US-7875615-B2,A61P1/16,"[hepatoprotective, disorders, gallbladder, cholagogues, litholytics, agents, liver]"
1048570,1048570,US-7875615-B2,A61P15/12,"[climacteric, disorders]"
1048573,1048573,US-7875615-B2,C07D249/08,"[hydrogenated, triazoles]"


In [41]:
pd.set_option('display.max_colwidth', None)
only_cpc_code

,idx,publication_number,cpc_code,cpc_desc
0,0,US-10000095-B2,B60C13/02,"[ribs, grooves, arrangement]"
1,1,US-10000095-B2,B60C15/0045,"[edges, bead, turn, ply, belt, extending, core, folded]"
2,2,US-10000095-B2,B60C15/0242,"[flange, position, rim, protectors, bead, located, outside, extensions, radially]"
3,3,US-10000095-B2,B60C17/0009,"[shaped, sidewall, crescent, comprising, inserts, rubber]"
5,5,US-10000095-B2,B60C2015/009,"[value, ratio, carcass, portion, proportion, defined, terminal, height, section, numerical, terms]"
...,...,...,...,...
1048567,1048567,US-7875427-B2,G01N33/6863,"[growth, gm, cytokines, cell, cnf, csf, biological, lymphotoxin, mif, response, proteins, modifying, differentiation, proliferation, immune, tnf, receptors]"
1048568,1048568,US-7875615-B2,A61P1/16,"[hepatoprotective, disorders, gallbladder, cholagogues, litholytics, agents, liver]"
1048570,1048570,US-7875615-B2,A61P15/12,"[climacteric, disorders]"
1048573,1048573,US-7875615-B2,C07D249/08,"[hydrogenated, triazoles]"


In [42]:
only_cpc_code = only_cpc_code.replace(',', ' ', regex=True)

In [43]:
only_cpc_code

,idx,publication_number,cpc_code,cpc_desc
0,0,US-10000095-B2,B60C13/02,[ribs grooves arrangement]
1,1,US-10000095-B2,B60C15/0045,[edges bead turn ply belt extending core folded]
2,2,US-10000095-B2,B60C15/0242,[flange position rim protectors bead located outside extensions radially]
3,3,US-10000095-B2,B60C17/0009,[shaped sidewall crescent comprising inserts rubber]
5,5,US-10000095-B2,B60C2015/009,[value ratio carcass portion proportion defined terminal height section numerical terms]
...,...,...,...,...
1048567,1048567,US-7875427-B2,G01N33/6863,[growth gm cytokines cell cnf csf biological lymphotoxin mif response proteins modifying differentiation proliferation immune tnf receptors]
1048568,1048568,US-7875615-B2,A61P1/16,[hepatoprotective disorders gallbladder cholagogues litholytics agents liver]
1048570,1048570,US-7875615-B2,A61P15/12,[climacteric disorders]
1048573,1048573,US-7875615-B2,C07D249/08,[hydrogenated triazoles]


In [44]:
pd.set_option('display.max_colwidth', None)
import pandas as pd
corrected_test_df=pd.DataFrame()
corrected_test_df =   pd.read_csv('/kaggle/input/corrected-square-test/corrected_square_test.csv')
corrected_test_df

,publication_number,cpc_desc
0,US-2017082634-A1,involving kinase
1,US-2017082634-A1,omics proteomics glycomics lipidomics methods analysis focusing entire complement classes biological molecules subsets thereof proteomes glycomes lipidomes
2,US-2017082634-A1,methods identifying protein interactions mixtures
3,US-2017082634-A1,methods protein analysis involving mass spectrometry
4,US-2017082634-A1,ict specially adapted hybridisation gene protein expression
5,US-2017082634-A1,gene protein expression profiling ratio estimation normalisation
6,US-2017082634-A1,ict specially adapted modelling simulations systems biology gene regulatory networks protein interaction metabolic
7,US-2017180470-A1,discovery centralising entities
8,US-2017180470-A1,supporting data block transmission mechanisms file transfer
9,US-2017180470-A1,distributed storage data networks transport arrangements network file nfs area san attached nas


In [45]:
schema = Schema(publication_number=TEXT(stored=True),
                cpc_desc=KEYWORD(stored=True))

In [46]:
import os
from whoosh import index

if not os.path.exists("cpcdescdir"):
    os.mkdir("cpcdescdir")

cpcdescidx = index.create_in("cpcdescdir", schema)

In [47]:
#ix = index.open_dir("cpcdescdir")

with cpcdescidx.writer() as writer:                                      

   for i in corrected_test_df.index:                                         
       writer.update_document(publication_number = str(corrected_test_df.loc[i,"publication_number"]), 
                              cpc_desc = str(corrected_test_df.loc[i, "cpc_desc"])) 

In [48]:
from whoosh import index
from whoosh.fields import Schema, TEXT, ID
from whoosh.qparser import QueryParser

# Define the schema for the index
schema = Schema(
    publication_number=TEXT(stored=True),
    cpc_desc=KEYWORD(stored=True)
)
pd.set_option('display.max_rows', 500)
# Path to the index directory
index_dir = "/kaggle/working/cpcdescdir"

# Open the index
ix = index.open_dir(index_dir)

tdf = pd.DataFrame(columns = ['publication_number', 'nearest_numbers'])
refdf = pd.DataFrame(columns = ['publication_number', 'cpc_desc'])
querydf = pd.DataFrame(columns = ['publication_number', 'query'])
dict_list = []
# Print the schema
print("Schema:")
for field_name, field_type in ix.schema.items():
    print(f"{field_name}: {field_type}")

# Create a searcher object
with ix.searcher() as searcher:
    # Define a query parser and parse a query
    query_parser = QueryParser("cpc_desc", ix.schema)
    query = query_parser.parse("*")  # This will match all documents

    # Search the index
    #results = searcher.search(query)
    results = searcher.search(query, limit=None)
    results_list = []
    pub_number_list = []
    cpc_desc_list = []
    seen_publication_numbers = set()
    
    # Print the results
    #print("\nIndex Contents:")
    for result in results:
        #print(dict(result))
        #print(result['publication_number']) 
        result_dict = dict(result)
        results_list.append(result_dict)
        pub_number = result_dict.get('publication_number')
        # Check if the publication number is already seen
        
        if pub_number not in seen_publication_numbers:
            #cpc_query = result_dict.get('cpc_desc')
            
            
            #seen_publication_numbers.add(cpc_query)
            #qdf1 = pd.DataFrame(seen_publication_numbers)
            seen_publication_numbers.add(pub_number)
            pub_number_list.append(result_dict.get('publication_number'))
            cpc_desc_list.append(result_dict.get('cpc_desc'))
            #qdf2 = result['publication_number']
            #querydf = pd.concat([querydf,qdf1 ], ignore_index=True)
            #print(seen_publication_numbers)                     
            #print(result_dict)
            #print(result['publication_number'])
            selection = (ttnearest_df.publication_number == result['publication_number'])
            filtered_df = ttnearest_df[selection].reset_index(drop=True)
            tdf = pd.concat([tdf, filtered_df], ignore_index=True)
            #print(ttnearest_df[selection])
            #row_dict = {'publication_number': result['publication_number'], 'nearest_numbers': 'ttnearest_df.nearest_numbers'}
            #new_row = {'publication_number':result['publication_number'], 'nearest_numbers': 'ttnearest_df.nearest_numbers' }
            #dict_list.append(row_dict)
            #tdf = tdf.append(new_row, ignore_index=True) 
            #tdf = pd.DataFrame.from_dict(dict_list)
            picking  = (corrected_test_df.publication_number == result['publication_number'])
            referred_df = corrected_test_df[picking].reset_index(drop=True)
            refdf = pd.concat([refdf, referred_df], ignore_index=True)
            #print(result['publication_number'])
results_df = pd.DataFrame(results_list)
unique_results_df = results_df.drop_duplicates(subset='publication_number', keep='first')
unique_results_df.rename(columns={'cpc_desc': 'query'}, inplace=True)
unique_results_df = unique_results_df[[ 'publication_number','query']]
#tdf.to_csv('final_test_results.csv')

#seen_publication_numbers

unique_results_df.reset_index(drop=True)


Schema:
cpc_desc: KEYWORD(format=Frequency(boost=1.0), scorable=False, stored=True, unique=False)
publication_number: TEXT(format=Positions(boost=1.0), scorable=True, stored=True, unique=None)


/tmp/ipykernel_33/2286608152.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_results_df.rename(columns={'cpc_desc': 'query'}, inplace=True)


,publication_number,query
0,US-2017082634-A1,involving kinase
1,US-2017180470-A1,discovery centralising entities
2,US-2018029544-A1,sunroofs precedence sliding roof
3,US-2022408153-A1,emotion mood input determined basis sensed human body parameters pulse heart rate beat temperature skin facial expressions iris voice pitch brain activity patterns
4,US-2268569-A,buckets discharging overhead container mounted machine
5,US-3371854-A,using gettering substances
6,US-3589189-A,characterised drive mechanism valves membrane index
7,US-3881203-A,symmetrical generally shaped clamps
8,US-4845770-A,sensing data fields affixed objects articles coded labels postal sorting conveying
9,US-695233-A,automatic carbide feed valves


In [49]:
results_list=[]
qp = QueryParser("content", schema=cpcdescidx.schema)
for index, row in unique_results_df.iterrows():
    query_str = row['query']
    q = qp.parse(query_str)
# Perform the search

    with cpcdescidx.searcher() as s:
        results = s.search(q)
        result_string = str(results)
        if isinstance(result_string, str):
            terms_pattern = re.compile(r"Term\('content', '([^']+)'\)")
            terms = terms_pattern.findall(result_string)
            if terms:
                formatted_terms = " AND ".join(f"'{term}'" for term in terms)
        
        
        results_list.append(formatted_terms)
        #print(results)
final_df = pd.DataFrame({
    'publication_number': pub_number_list,
    'cpc_desc': results_list
})

results_list
final_df['final'] = final_df['publication_number'] + '\xa0' * 5 + final_df['cpc_desc']
pd.set_option('display.max_colwidth', None)
print(final_df['final'].to_string(index=False))

                                                                                                                                                                                                                                                                                 US-2017082634-A1     'involving' AND 'kinase'
                                                                                                                                                                                                                                                            US-2017180470-A1     'discovery' AND 'centralising' AND 'entities'
                                                                                                                                                                                                                                                     US-2018029544-A1     'sunroofs' AND 'precedence' AND 'sliding' AND 'roof'
US-2022408153-A1     'emotion' AND 'mood' A

In [50]:
new_final_df=final_df.rename(columns = {'cpc_desc': 'query'})
new_final_df
del new_final_df['final']
new_final_df.reset_index(drop=True, inplace=True) 
blankIndex=[''] * len(new_final_df)
new_final_df.index=blankIndex
new_final_df

,publication_number,query
,US-2017082634-A1,'involving' AND 'kinase'
,US-2017180470-A1,'discovery' AND 'centralising' AND 'entities'
,US-2018029544-A1,'sunroofs' AND 'precedence' AND 'sliding' AND 'roof'
,US-2022408153-A1,'emotion' AND 'mood' AND 'input' AND 'determined' AND 'basis' AND 'sensed' AND 'human' AND 'body' AND 'parameters' AND 'pulse' AND 'heart' AND 'rate' AND 'beat' AND 'temperature' AND 'skin' AND 'facial' AND 'expressions' AND 'iris' AND 'voice' AND 'pitch' AND 'brain' AND 'activity' AND 'patterns'
,US-2268569-A,'buckets' AND 'discharging' AND 'overhead' AND 'container' AND 'mounted' AND 'machine'
,US-3371854-A,'using' AND 'gettering' AND 'substances'
,US-3589189-A,'characterised' AND 'drive' AND 'mechanism' AND 'valves' AND 'membrane' AND 'index'
,US-3881203-A,'symmetrical' AND 'generally' AND 'shaped' AND 'clamps'
,US-4845770-A,'sensing' AND 'data' AND 'fields' AND 'affixed' AND 'objects' AND 'articles' AND 'coded' AND 'labels' AND 'postal' AND 'sorting' AND 'conveying'
,US-695233-A,'automatic' AND 'carbide' AND 'feed' AND 'valves'


In [51]:
new_final_df.to_csv("submission.csv")
new_final_df

,publication_number,query
,US-2017082634-A1,'involving' AND 'kinase'
,US-2017180470-A1,'discovery' AND 'centralising' AND 'entities'
,US-2018029544-A1,'sunroofs' AND 'precedence' AND 'sliding' AND 'roof'
,US-2022408153-A1,'emotion' AND 'mood' AND 'input' AND 'determined' AND 'basis' AND 'sensed' AND 'human' AND 'body' AND 'parameters' AND 'pulse' AND 'heart' AND 'rate' AND 'beat' AND 'temperature' AND 'skin' AND 'facial' AND 'expressions' AND 'iris' AND 'voice' AND 'pitch' AND 'brain' AND 'activity' AND 'patterns'
,US-2268569-A,'buckets' AND 'discharging' AND 'overhead' AND 'container' AND 'mounted' AND 'machine'
,US-3371854-A,'using' AND 'gettering' AND 'substances'
,US-3589189-A,'characterised' AND 'drive' AND 'mechanism' AND 'valves' AND 'membrane' AND 'index'
,US-3881203-A,'symmetrical' AND 'generally' AND 'shaped' AND 'clamps'
,US-4845770-A,'sensing' AND 'data' AND 'fields' AND 'affixed' AND 'objects' AND 'articles' AND 'coded' AND 'labels' AND 'postal' AND 'sorting' AND 'conveying'
,US-695233-A,'automatic' AND 'carbide' AND 'feed' AND 'valves'
